# Lesson 3: Agentic Search

In [1]:
# libraries
from dotenv import load_dotenv
import os
from tavily import TavilyClient

# load environment variables from .env file
_ = load_dotenv()

# connect
client = TavilyClient(api_key=os.environ.get("TAVILY_API_KEY"))

In [2]:
# run search
result = client.search("What is in Nvidia's new Blackwell GPU?",
                       include_answer=True)

# print the answer
result["answer"]


'The Blackwell GPU features fifth-generation Tensor Cores, Transformer Engine, and NVLink interconnect. It includes advanced security and supports large AI models. It was announced in March 2024.'

## Regular search

In [3]:
# choose location (try to change to your own city!)

city = "San Francisco"

query = f"""
    what is the current weather in {city}?
    Should I travel there today?
    "weather.com"
"""

> Note: search was modified to return expected results in the event of an exception. High volumes of student traffic sometimes cause rate limit exceptions.

In [4]:
import requests
from bs4 import BeautifulSoup
from duckduckgo_search import DDGS
import re

ddg = DDGS()

def search(query, max_results=6):
    try:
        results = ddg.text(query, max_results=max_results)
        return [i["href"] for i in results]
    except Exception as e:
        print(f"returning previous results due to exception reaching ddg.")
        results = [ # cover case where DDG rate limits due to high deeplearning.ai volume
            "https://weather.com/weather/today/l/USCA0987:1:US",
            "https://weather.com/weather/hourbyhour/l/54f9d8baac32496f6b5497b4bf7a277c3e2e6cc5625de69680e6169e7e38e9a8",
        ]
        return results  


for i in search(query):
    print(i)

https://weather.com/weather/today/l/San+Francisco+CA+USCA0987:1:US


In [5]:
def scrape_weather_info(url):
    """Scrape content from the given URL"""
    if not url:
        return "Weather information could not be found."
    
    # fetch data
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        return "Failed to retrieve the webpage."

    # parse result
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup


> Note: This produces a long output, you may want to right click and clear the cell output after you look at it briefly to avoid scrolling past it.

In [6]:
# use DuckDuckGo to find websites and take the first result
url = search(query)[0]

# scrape first wesbsite
soup = scrape_weather_info(url)

print(f"Website: {url}\n\n")
print(str(soup.body)[:50000]) # limit long outputs

Website: https://weather.com/weather/today/l/San+Francisco+CA+USCA0987:1:US


<body><div class="appWrapper DaybreakLargeScreen LargeScreen lightTheme twcTheme DaybreakLargeScreen--appWrapper--ZkDop" id="appWrapper"><div class="region-sidebarNavigation regionSidebarNavigation"><div class="removeIfEmpty" id="WxuSidebarNavigation-sidebarNavigation-sidebar-nav-injection"><div aria-hidden="false" class="SidebarNavigation--sidebarWrapper--v+g78"><nav aria-label="Site Navigation Links" class="SidebarNavigation--sidebar--A5yHr" data-testid="sidebar-navigation"><div class="SidebarNavigation--sidebarHeader--bMAxo"><button class="SidebarNavigation--overlayControlButton--cSFvn openSidebarOverlayButton" tabindex="0" type="button"><span class="Icon--iconWrapper--vSeDL"><svg class="Icon--icon--ySD-o" fill="none" name="menu" viewbox="0 0 48 48" xmlns="http://www.w3.org/2000/svg">
<title>Menu</title>
<path d="M42 13.714c0-.947-.672-1.714-1.5-1.714h-33c-.828 0-1.5.768-1.5 1.714 0 .947.672 1.715 1.5 1.71

In [7]:
# extract text
weather_data = []
for tag in soup.find_all(['h1', 'h2', 'h3', 'p']):
    text = tag.get_text(" ", strip=True)
    weather_data.append(text)

# combine all elements into a single string
weather_data = "\n".join(weather_data)

# remove all spaces from the combined text
weather_data = re.sub(r'\s+', ' ', weather_data)
    
print(f"Website: {url}\n\n")
print(weather_data)

Website: https://weather.com/weather/today/l/San+Francisco+CA+USCA0987:1:US


Recent Locations Weather Forecasts Radar & Maps News & Media Products & Account Lifestyle Specialty Forecasts Downtown, San Francisco, California Weather Today in Downtown, San Francisco, California 7:11 am 6:41 pm Hourly Weather - Downtown, San Francisco, California Now Partly Cloudy 7 pm Partly Cloudy 8 pm Partly Cloudy 9 pm Partly Cloudy Don't Miss Seasonal Hub 10 Day Weather - Downtown, San Francisco, California Tonight Night Partly cloudy skies this evening will give way to occasional showers overnight. Low 59F. Winds SSW at 10 to 20 mph. Chance of rain 40%. Thu 09 Day Partly to mostly cloudy. High 68F. Winds SSW at 10 to 15 mph. Night Partly cloudy. Low around 55F. Winds W at 10 to 15 mph. Fri 10 Day Mostly sunny. High 67F. Winds WSW at 10 to 15 mph. Night Partly cloudy skies. Low 56F. Winds WNW at 10 to 15 mph. Sat 11 Day Sunny. High 67F. Winds WNW at 10 to 20 mph. Night Clear skies. Low 53F. Winds NW 

## Agentic Search

In [8]:
# run search
result = client.search(query, max_results=1)

# print first result
data = result["results"][0]["content"]

print(data)

{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1759968586, 'localtime': '2025-10-08 17:09'}, 'current': {'last_updated_epoch': 1759968000, 'last_updated': '2025-10-08 17:00', 'temp_c': 21.1, 'temp_f': 70.0, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 17.0, 'wind_kph': 27.4, 'wind_degree': 222, 'wind_dir': 'SW', 'pressure_mb': 1011.0, 'pressure_in': 29.84, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 59, 'cloud': 75, 'feelslike_c': 21.1, 'feelslike_f': 70.0, 'windchill_c': 19.9, 'windchill_f': 67.9, 'heatindex_c': 20.5, 'heatindex_f': 68.9, 'dewpoint_c': 13.2, 'dewpoint_f': 55.8, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 0.9, 'gust_mph': 21.9, 'gust_kph': 35.2}}


In [9]:
import json
from pygments import highlight, lexers, formatters

# parse JSON
parsed_json = json.loads(data.replace("'", '"'))

# pretty print JSON with syntax highlighting
formatted_json = json.dumps(parsed_json, indent=4)
colorful_json = highlight(formatted_json,
                          lexers.JsonLexer(),
                          formatters.TerminalFormatter())

print(colorful_json)


{
    "location": {
        "name": "San Francisco",
        "region": "California",
        "country": "United States of America",
        "lat": 37.775,
        "lon": -122.4183,
        "tz_id": "America/Los_Angeles",
        "localtime_epoch": 1759968586,
        "localtime": "2025-10-08 17:09"
    },
    "current": {
        "last_updated_epoch": 1759968000,
        "last_updated": "2025-10-08 17:00",
        "temp_c": 21.1,
        "temp_f": 70.0,
        "is_day": 1,
        "condition": {
            "text": "Partly cloudy",
            "icon": "//cdn.weatherapi.com/weather/64x64/day/116.png",
            "code": 1003
        },
        "wind_mph": 17.0,
        "wind_kph": 27.4,
        "wind_degree": 222,
        "wind_dir": "SW",
        "pressure_mb": 1011.0,
        "pressure_in": 29.84,
        "precip_mm": 0.0,
        "precip_in": 0.0,
        "humidity": 59,
        "cloud": 75,
        "feelslike_c": 21.1,
        "feelslike_f": 70.0,
        "windchill_c": 19.9,
    

<img src="./google_sample.png" width="800" height="600">